In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields)

In [3]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  734.0


In [4]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  208.0


In [5]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [6]:
print('Huelva (capital)\n')
print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Confirmados PCR 7 días x 1,000 habitantes:  0.07657
Un positivo PCR cada 13060 personas en los últimos 7 días 

Confirmados PCR 14 días x 1,000 habitantes:  0.17402
Un positivo PCR cada 5747 personas en los últimos 14 días


In [7]:
filtro = ((df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [11]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR 14 días,Confirmados PCR 7 días,Población,tasa7,tasa14
Lugar de residencia,,,,,
Corteconcepción,2.0,2.0,548.0,364.963504,364.963504
Villalba del Alcor,12.0,12.0,3338.0,359.496705,359.496705
Minas de Riotinto,9.0,8.0,3848.0,207.900208,233.887734
Cabezas Rubias,1.0,1.0,730.0,136.986301,136.986301
Escacena del Campo,3.0,3.0,2284.0,131.348511,131.348511
Paterna del Campo,3.0,2.0,3509.0,56.996295,85.494443
Lepe,18.0,9.0,27431.0,32.809595,65.619190
Aljaraque,13.0,3.0,21260.0,14.111007,61.147695
Isla Cristina,13.0,5.0,21264.0,23.513920,61.136193
